*Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов.  
Даны значения роста в трех группах случайно выбранных спортсменов:  
Футболисты: 173, 175, 180, 178, 177, 185, 183, 182.  
Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180.  
Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170.*

In [3]:
import numpy as np
import scipy.stats as stats
soccer = np.array([173, 175, 180, 178, 177, 185, 183, 182])
hockey=np.array([177, 179, 180, 188, 177, 172, 171, 184, 180])
powerlift = np.array([172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170])

Проверим условия применимости дисперсионного анализа:  
1)нормальность

In [4]:
stats.shapiro(soccer)

ShapiroResult(statistic=0.9775082468986511, pvalue=0.9495404362678528)

In [5]:
stats.shapiro(hockey)

ShapiroResult(statistic=0.9579196572303772, pvalue=0.7763139009475708)

In [6]:
stats.shapiro(powerlift)

ShapiroResult(statistic=0.9386808276176453, pvalue=0.5051165223121643)

Все pvalue больше 0.05, что говорит о верности нулевой гипотезы о нормальном распределении выборок

2)Однородность дисперсий
По Бартлетту

In [7]:
stats.bartlett(soccer,hockey,powerlift)

BartlettResult(statistic=0.4640521043406442, pvalue=0.7929254656083131)

Большой pvalue говорит о том, что тестируемые выборки имеют схожие параметры дисперсии.  

Заодно проверим размеры выборок:

In [9]:
len(soccer) == len(hockey) == len(powerlift)

False

Размеры разные, нужно будет это учесть при составлении датасета для post hoc теста


Условие нормальности, разнородности выполняется. Условие о независимости измерений будет считать за выполенное (доверяем источнику).  
Таким образом, можем продолжать наш дисперсионный анализ. За количественную характеристику будут приниматься измерения роста, а за качественную - категории спортсменов.

Для начала проверим наличие статистических различий между группами по криттерию Фишера

In [8]:
stats.f_oneway(soccer,hockey,powerlift)

F_onewayResult(statistic=5.500053450812596, pvalue=0.010482206918698693)

pvalue меньше $\alpha = 5 \%$ , значит верна альтернативная гипотеза о различии между выборками.

Теперь попарно сравним выборки через **post hoc test** Тьюки

In [19]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
values = np.concatenate([soccer,hockey,powerlift])
categ = np.repeat(['soccer','hockey','powerlift'],repeats=[len(soccer),len(hockey),len(powerlift)])
tukey_result = pairwise_tukeyhsd(endog = values,groups = categ, alpha=0.05)
print(tukey_result)


    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower    upper  reject
-----------------------------------------------------------
   hockey powerlift  -5.9394 0.0284 -11.3181 -0.5607   True
   hockey    soccer   0.4583  0.979  -5.3566  6.2732  False
powerlift    soccer   6.3977 0.0219   0.8372 11.9583   True
-----------------------------------------------------------


Нулевая гипотеза принимается только в одной паре в силу большой p-value : хоккеисты и футболисты не имеет статистических различий и имеют одинаковое мат.ожидание.  
Выведем доверительные интервалы для отличающихся пар выборок.


In [26]:
cat_names = np.unique(categ)
pair1_av = np.mean(np.hstack((hockey,powerlift)))
print(f"Доверительный интервал мат.ожидания c надежностью в 95% для пары {cat_names[0]} и {cat_names[1]}: \n")
print(f"{tukey_result.confint[0]+pair1_av}")

Доверительный интервал мат.ожидания c надежностью в 95% для пары hockey и powerlift: 

[164.08186306 174.83934906]


In [30]:
pair3_av = np.mean(np.hstack((powerlift,soccer)))
print(f"Доверительный интервал мат.ожидания c надежностью в 95% для пары {cat_names[1]} и {cat_names[2]}: \n")
print(f"{tukey_result.confint[2]+pair3_av}")

Доверительный интервал мат.ожидания c надежностью в 95% для пары powerlift и soccer: 

[176.25821573 187.37934408]


In [27]:
#более ручной метод среднего , что правильней?
#(np.mean(hockey)+np.mean(powerlift))/2